### Change Runtime type to **GPU**

In [ ]:
#installation
from google.colab import drive
drive.mount('/content/drive')

### Change your **Google drive path** then just click run

In [ ]:
# CHANGE YOUR DRIVE PATH HERE
%cd /content/drive/MyDrive/YOUR_FOLDER_THAT_HAVE_RAW_DATA_FOLDER
!pip install -r requirement.txt

In [ ]:
#import
import os
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torchvision import transforms as T
from helper import device, ToJPG, predict, ToTensor, ToTensorShowImage, plot_predict_most_confs, plot_predict

In [ ]:
#load trained model
current_path = os.getcwd()
folder_name = current_path.split('/')[-1]
%cd ..
for file in os.listdir(folder_name) :
    if file.endswith('pt') :
        weight_path = file
os.chdir(folder_name)
class_names = [line.strip() for line in open('class_names.txt', 'r')]

os.makedirs('predict_images', exist_ok=True)
trained_model = torchvision.models.efficientnet_b2().to(device)
trained_model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.3, inplace=True), 
    torch.nn.Linear(in_features=1408, 
                    out_features=len(class_names), 
                    bias=True)).to(device)
trained_model.load_state_dict(torch.load(f=weight_path, map_location=device))

### upload predict images in **predict_images** folder
- !!! uploaded images before run the next cell
- click refresh if you don't see the folder

In [ ]:
# to .jpg format
for image_name in os.listdir('predict_images') :
    image_path = os.path.join('predict_images', image_name)
    ToJPG(image_path)

### predict a single image
- change image_path 

In [ ]:
image_path = 'predict_images/YOUR_IMAGE_NAME.jpg' 
most_like = 3 # find most like 3 classes

pred_class, confs = predict(ToTensor(image_path), trained_model, class_names)
confs = confs*100
print(f'''
predict class : {pred_class}
confident : {np.max(confs):.2f}%
''')

In [ ]:
plot_predict_most_confs(image_path, most_like, class_names, trained_model)

### predict multiple images
- just run and see a result

In [ ]:
images_path = [os.path.join('predict_images', image_name) for image_name in os.listdir('predict_images') if not image_name.endswith('avif')]
size = int(len(images_path)**(1/2)) + 1
plt.figure(figsize=(4*size, 4*size))
for i, image_name in enumerate(images_path, start=1) : 
    plt.subplot(size, size, i)
    image_fp = ToTensor(image_name)
    image_fs = ToTensorShowImage(image_name)
    plot_predict(image_fp, trained_model, image_fs, class_names, color='black')